<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/exchangerate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest

# Helper for Money comparison, similar to C++ close
def close_money(m1, m2, tolerance=1e-9):
    if m1.currency() != m2.currency():
        return False
    return ql.close_enough(m1.value(), m2.value(), tolerance)

class ExchangeRateTests(unittest.TestCase):

    def setUp(self):
        # Ensure clean state for ExchangeRateManager tests
        ql.ExchangeRateManager.instance().clear()
        # Set default conversion type for Money for these tests
        self.original_conversion_type = ql.Money.Settings.instance().conversionType
        ql.Money.Settings.instance().conversionType = ql.Money.NoConversion

    def tearDown(self):
        # Restore original conversion type
        ql.Money.Settings.instance().conversionType = self.original_conversion_type
        ql.ExchangeRateManager.instance().clear() # Clean up after each test


    def test_direct_exchange_rates(self): # Renamed from testDirect
        print("Testing direct exchange rates...")
        eur = ql.EURCurrency()
        usd = ql.USDCurrency()

        eur_usd_rate = ql.ExchangeRate(eur, usd, 1.2042)

        m1_eur = ql.Money(50000.0, eur)
        m2_usd = ql.Money(100000.0, usd)

        calculated1 = eur_usd_rate.exchange(m1_eur)
        expected1 = ql.Money(m1_eur.value() * eur_usd_rate.rate(), usd)
        self.assertTrue(close_money(calculated1, expected1),
                        f"Direct EUR to USD: Expected {expected1}, Got {calculated1}")

        calculated2 = eur_usd_rate.exchange(m2_usd)
        expected2 = ql.Money(m2_usd.value() / eur_usd_rate.rate(), eur)
        self.assertTrue(close_money(calculated2, expected2),
                        f"Direct USD to EUR: Expected {expected2}, Got {calculated2}")

    def test_derived_exchange_rates(self): # Renamed from testDerived
        print("Testing derived exchange rates...")
        eur = ql.EURCurrency()
        usd = ql.USDCurrency()
        gbp = ql.GBPCurrency()

        eur_usd_rate_obj = ql.ExchangeRate(eur, usd, 1.2042)
        eur_gbp_rate_obj = ql.ExchangeRate(eur, gbp, 0.6612)

        # Derived rate: USD per GBP (via EUR)
        # chain(EUR/USD, EUR/GBP) gives USD/GBP if EUR is common source
        # chain(USD/EUR, EUR/GBP) gives USD/GBP
        # chain(EUR/USD, GBP/EUR) gives GBP/USD
        # The C++ test implies derived = USD/GBP (target of first / target of second, common source)
        # ExchangeRate::chain(r1, r2)
        # If r1 = A/B, r2 = A/C, then chain gives B/C (r1.rate / r2.rate)
        # If r1 = A/B, r2 = C/A, then chain gives C/B (1 / (r1.rate * r2.rate))
        # Here, r1 = EUR/USD (source EUR, target USD), r2 = EUR/GBP (source EUR, target GBP)
        # Chain should result in USD/GBP with rate = (rate EUR/USD) / (rate EUR/GBP) if USD is new source
        # OR GBP/USD with rate = (rate EUR/GBP) / (rate EUR/USD) if GBP is new source
        # The C++ ExchangeRate::chain logic determines source/target.
        # Let's verify what QL Python's chain does.
        # If r1.target == r2.target, then source1/source2 or source2/source1.
        # If r1.source == r2.source, then target1/target2 or target2/target1.
        # Example: r1=EUR/USD, r2=EUR/GBP. Common source EUR. Result can be USD/GBP or GBP/USD.
        # QL C++ logic: if (r1.source == r2.source) return ExchangeRate(r2.target, r1.target, r1.rate/r2.rate);
        # So, GBP/USD with rate = (EUR/USD rate) / (EUR/GBP rate) = 1.2042 / 0.6612
        derived_rate = ql.ExchangeRate.chain(eur_usd_rate_obj, eur_gbp_rate_obj)
        # Expected: derived_rate.source() == gbp, derived_rate.target() == usd
        self.assertEqual(derived_rate.source(), gbp)
        self.assertEqual(derived_rate.target(), usd)


        m1_gbp = ql.Money(50000.0, gbp)
        m2_usd = ql.Money(100000.0, usd)

        # Convert m1_gbp (GBP) using derived_rate (GBP/USD)
        calculated1 = derived_rate.exchange(m1_gbp) # Should give USD
        # Expected rate for GBP/USD is eur_usd_rate.rate() / eur_gbp_rate_obj.rate()
        # So, value_gbp * (rate_eur_usd / rate_eur_gbp) USD
        expected_value1 = m1_gbp.value() * (eur_usd_rate_obj.rate() / eur_gbp_rate_obj.rate())
        expected1 = ql.Money(expected_value1, usd)
        self.assertTrue(close_money(calculated1, expected1),
                        f"Derived GBP to USD: Expected {expected1}, Got {calculated1}")

        # Convert m2_usd (USD) using derived_rate (GBP/USD)
        calculated2 = derived_rate.exchange(m2_usd) # Should give GBP
        # Expected rate for USD/GBP is (rate_eur_gbp / rate_eur_usd)
        # So, value_usd * (rate_eur_gbp / rate_eur_usd) GBP
        expected_value2 = m2_usd.value() * (eur_gbp_rate_obj.rate() / eur_usd_rate_obj.rate())
        expected2 = ql.Money(expected_value2, gbp)
        self.assertTrue(close_money(calculated2, expected2),
                        f"Derived USD to GBP: Expected {expected2}, Got {calculated2}")

    def test_direct_lookup(self):
        print("Testing lookup of direct exchange rates...")
        rate_manager = ql.ExchangeRateManager.instance()
        # rate_manager.clear() # Done in setUp

        eur = ql.EURCurrency(); usd = ql.USDCurrency()
        date1 = ql.Date(4, ql.August, 2004)
        date2 = ql.Date(5, ql.August, 2004)

        eur_usd_d1_direct = ql.ExchangeRate(eur, usd, 1.1983) # EUR/USD = 1.1983
        eur_usd_d2_inverted = ql.ExchangeRate(usd, eur, 1.0/1.2042) # USD/EUR = 1/1.2042 => EUR/USD = 1.2042

        rate_manager.add(eur_usd_d1_direct, date1)
        rate_manager.add(eur_usd_d2_inverted, date2)

        m1_eur = ql.Money(50000.0, eur)
        m2_usd = ql.Money(100000.0, usd)

        # Lookup EUR/USD for date1
        eur_usd_lookup_d1 = rate_manager.lookup(eur, usd, date1, ql.ExchangeRate.Direct)
        self.assertAlmostEqual(eur_usd_lookup_d1.rate(), eur_usd_d1_direct.rate())
        calculated = eur_usd_lookup_d1.exchange(m1_eur)
        expected = ql.Money(m1_eur.value() * eur_usd_d1_direct.rate(), usd)
        self.assertTrue(close_money(calculated, expected), "Direct lookup EUR/USD date1")

        # Lookup EUR/USD for date2
        eur_usd_lookup_d2 = rate_manager.lookup(eur, usd, date2, ql.ExchangeRate.Direct)
        # eur_usd_d2_inverted was USD/EUR = 1/1.2042. So EUR/USD lookup should give 1.2042.
        self.assertAlmostEqual(eur_usd_lookup_d2.rate(), 1.0 / eur_usd_d2_inverted.rate())
        calculated = eur_usd_lookup_d2.exchange(m1_eur)
        expected = ql.Money(m1_eur.value() * (1.0 / eur_usd_d2_inverted.rate()), usd)
        self.assertTrue(close_money(calculated, expected), "Direct lookup EUR/USD date2")

        # ... similar checks for USD/EUR lookups ...


    def test_triangulated_lookup(self):
        print("Testing lookup of triangulated exchange rates...")
        rate_manager = ql.ExchangeRateManager.instance()
        eur = ql.EURCurrency(); usd = ql.USDCurrency(); itl = ql.ITLCurrency() # Italian Lira

        # ITL is an EMU currency, so EUR/ITL is fixed at 1936.27
        # This fixed rate is known to ExchangeRateManager by default.

        date1 = ql.Date(4, ql.August, 2004)
        date2 = ql.Date(5, ql.August, 2004)
        eur_usd_d1 = ql.ExchangeRate(eur, usd, 1.1983)
        eur_usd_d2 = ql.ExchangeRate(eur, usd, 1.2042)
        rate_manager.add(eur_usd_d1, date1)
        rate_manager.add(eur_usd_d2, date2)

        m1_itl = ql.Money(50000000.0, itl)

        # Lookup ITL/USD for date1 (triangulates via EUR: ITL -> EUR -> USD)
        # Rate ITL/USD = (Rate ITL/EUR) * (Rate EUR/USD)
        # Rate ITL/EUR = 1 / 1936.27
        itl_usd_lookup_d1 = rate_manager.lookup(itl, usd, date1) # Smart lookup, will use triangulation
        calculated = itl_usd_lookup_d1.exchange(m1_itl)
        expected_rate_itl_usd_d1 = (1.0 / 1936.27) * eur_usd_d1.rate()
        expected = ql.Money(m1_itl.value() * expected_rate_itl_usd_d1, usd)
        self.assertTrue(close_money(calculated, expected), "Triangulated ITL/USD date1")
        self.assertAlmostEqual(itl_usd_lookup_d1.rate(), expected_rate_itl_usd_d1)

        # ... similar checks for date2 and USD/ITL ...

    def test_smart_lookup_chains(self): # Renamed from testSmartLookup
        print("Testing lookup of derived (chained) exchange rates...")
        rate_manager = ql.ExchangeRateManager.instance()
        eur=ql.EURCurrency(); usd=ql.USDCurrency(); gbp=ql.GBPCurrency()
        chf=ql.CHFCurrency(); sek=ql.SEKCurrency(); jpy=ql.JPYCurrency()

        date1 = ql.Date(4,ql.August,2004)
        date2 = ql.Date(5,ql.August,2004)

        # Add rates (some direct, some inverted to test manager's handling)
        rate_manager.add(ql.ExchangeRate(eur, usd, 1.1983), date1)    # EUR/USD
        rate_manager.add(ql.ExchangeRate(usd, eur, 1.0/1.2042), date2)# USD/EUR
        rate_manager.add(ql.ExchangeRate(gbp, eur, 1.0/0.6596), date1)# GBP/EUR
        rate_manager.add(ql.ExchangeRate(eur, gbp, 0.6612), date2)    # EUR/GBP
        rate_manager.add(ql.ExchangeRate(usd, chf, 1.2847), date1)    # USD/CHF
        rate_manager.add(ql.ExchangeRate(chf, usd, 1.0/1.2774), date2)# CHF/USD
        rate_manager.add(ql.ExchangeRate(sek, chf, 0.1674), date1)    # SEK/CHF
        rate_manager.add(ql.ExchangeRate(chf, sek, 1.0/0.1677), date2)# CHF/SEK
        rate_manager.add(ql.ExchangeRate(sek, jpy, 14.5450), date1)   # SEK/JPY
        rate_manager.add(ql.ExchangeRate(jpy, sek, 1.0/14.6110), date2)# JPY/SEK

        m_usd = ql.Money(100000.0, usd)
        m_eur = ql.Money(100000.0, eur)
        # ... other money amounts ...

        # Two-rate chain: USD -> SEK (via CHF: USD->CHF->SEK) for date1
        # USD/SEK = (USD/CHF) * (CHF/SEK)
        # CHF/SEK on date1 is 1 / (SEK/CHF rate) = 1 / 0.1674
        # So, USD/SEK = (USD/CHF rate) / (SEK/CHF rate) = 1.2847 / 0.1674
        usd_sek_d1_lookup = rate_manager.lookup(usd, sek, date1)
        expected_rate_usd_sek_d1 = (1.2847 / 0.1674)
        self.assertAlmostEqual(usd_sek_d1_lookup.rate(), expected_rate_usd_sek_d1)
        calculated = usd_sek_d1_lookup.exchange(m_usd)
        expected = ql.Money(m_usd.value() * expected_rate_usd_sek_d1, sek)
        self.assertTrue(close_money(calculated, expected), "Smart lookup USD/SEK date1 (2-chain)")

        # Three-rate chain: EUR -> SEK (via USD, CHF: EUR->USD->CHF->SEK) for date1
        # EUR/SEK = (EUR/USD) * (USD/CHF) * (CHF/SEK)
        # EUR/SEK = (EUR/USD rate) * (USD/CHF rate) / (SEK/CHF rate)
        #         = 1.1983 * 1.2847 / 0.1674
        eur_sek_d1_lookup = rate_manager.lookup(eur, sek, date1)
        expected_rate_eur_sek_d1 = (1.1983 * 1.2847 / 0.1674)
        self.assertAlmostEqual(eur_sek_d1_lookup.rate(), expected_rate_eur_sek_d1)
        calculated_eur_sek = eur_sek_d1_lookup.exchange(m_eur)
        expected_eur_sek = ql.Money(m_eur.value() * expected_rate_eur_sek_d1, sek)
        self.assertTrue(close_money(calculated_eur_sek, expected_eur_sek), "Smart lookup EUR/SEK date1 (3-chain)")

        # ... More chain tests for date2 and other pairs as in C++ ...


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)